# CGH ER ARRIVAL PREDICTION MODEL

#### Temporal exogenous SARIMAX model
#### Training period 2022-01-01 to 2024-12-31
#### Testing period 2025-01-01 to 2025-09-30

In [1]:
# Importing lib
import os
import re
import pandas as pd
import numpy as np
import time, gc, warnings
import pickle
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from sklearn.metrics import mean_squared_error, mean_absolute_error
import fastf1

In [2]:
# Suppress warning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.simplefilter("ignore", ConvergenceWarning)

In [3]:
# Config
CGH_ER_ARRIVALS_CSV_PATH = "AnE_Data_fin.csv"
CGH_ER_ARRIVALS_2018_2024_CSV_PATH = "edarrivals_20182024.csv"
CGH_ER_ARRIVALS_2025_CSV_PATH = "edarrivals_2025.csv"
CGH_CALENDAR_CSV_PATH = "calendar_2018_2026.csv"

CGH_TRAIN_START_DATE = "2022-01-01"
CGH_TRAIN_END_DATE   = "2024-12-31"
CGH_TEST_START_DATE  = "2025-01-01"
CGH_TEST_END_DATE    = "2025-09-30"

CGH_FULL_START_DATE = CGH_TRAIN_START_DATE
CGH_FULL_END_DATE   = CGH_TEST_END_DATE

CGH_FORECAST_HOURS = [24, 48, 72]
CGH_FORECAST_HORIZON_DAYS = [1, 2, 3]
CGH_WEEKLY_SEASONAL_PERIOD = 7

# Output names
CGH_MULTI_HORIZON_CSV = "CGH_ER_MultiHorizon_Predictions_Daily_Train2022to2024_Test2025_JnToSep.csv"
CGH_PROD_MODEL_PKL    = "CGH_ER_SARIMAX_TemporalExog_ProdModel_FullFit_2022to2025.pkl"

In [4]:
#  PUBLIC HOLIDAY FLAG USING CALENDAR CSV
cgh_full_index = pd.date_range(
    start=CGH_FULL_START_DATE,
    end=CGH_FULL_END_DATE,
    freq="D"
)
cgh_full_index = pd.to_datetime(cgh_full_index).tz_localize(None).normalize()

def load_cgh_calendar_df(calendar_csv_path: str) -> pd.DataFrame:
    cal = pd.read_csv(calendar_csv_path, low_memory=False)

    date_col_candidates = [c for c in cal.columns if str(c).strip().lower() in ("date", "ds")]
    if not date_col_candidates:
        raise ValueError("calendar_2018_2026.csv must have a Date or ds column.")

    cal = cal.rename(columns={date_col_candidates[0]: "Date"})
    cal["Date"] = pd.to_datetime(cal["Date"], dayfirst=True, errors="coerce")

    cal = cal.dropna(subset=["Date"]).sort_values("Date").set_index("Date")
    return cal

def build_cgh_public_holiday_flag_from_calendar(
    full_index: pd.DatetimeIndex,
    calendar_df: pd.DataFrame
) -> pd.Series:
    full_index = pd.to_datetime(full_index).tz_localize(None).normalize()

    preferred_cols = [
        c for c in calendar_df.columns
        if str(c).strip().lower() in (
            "is_public_holiday",
            "public_holiday",
            "is_ph",
            "ph",
            "holiday_flag",
            "is_holiday"
        )
    ]

    if preferred_cols:
        holiday_col = preferred_cols[0]
    else:
        possible_cols = [
            c for c in calendar_df.columns
            if ("holiday" in str(c).lower()) or ("public" in str(c).lower())
        ]
        if not possible_cols:
            return pd.Series(0, index=full_index, dtype=int, name="holiday")
        holiday_col = possible_cols[0]

    s = calendar_df[holiday_col]

    if pd.api.types.is_bool_dtype(s):
        s = s.astype(int)
    elif pd.api.types.is_numeric_dtype(s):
        s = s.fillna(0).astype(int)
    else:
        s = (
            s.astype(str).str.strip().str.lower()
            .map({
                "1": 1, "true": 1, "yes": 1, "y": 1, "t": 1,
                "holiday": 1, "public holiday": 1, "ph": 1
            })
            .fillna(0)
            .astype(int)
        )

    out = s.reindex(full_index).fillna(0).astype(int)
    out.name = "holiday"
    return out

calendar_df = load_cgh_calendar_df(CGH_CALENDAR_CSV_PATH)
cgh_public_holiday_flag = build_cgh_public_holiday_flag_from_calendar(
    cgh_full_index, calendar_df
)

# OTHER PUBLIC HOLIDAYS: POST +1 to +2 ONLY
def build_cgh_holiday_post_window(
    full_index: pd.DatetimeIndex,
    holiday_flag: pd.Series,
    post_days: int = 2
) -> pd.Series:
    hol = holiday_flag.reindex(full_index).fillna(0).astype(int)
    post = pd.Series(0, index=full_index, dtype=int)

    hol_pos = np.where(hol.values == 1)[0]
    for p in hol_pos:
        start = p + 1
        end   = min(p + post_days, len(post) - 1)
        if start <= end:
            post.iloc[start:end + 1] = 1

    post.name = "holiday_post_window_2d"
    return post

cgh_holiday_post_window_2d = build_cgh_holiday_post_window(
    cgh_full_index, cgh_public_holiday_flag, post_days=2
)

# CNY DETECTION (2 consecutive PH days)
def build_cgh_cny_flag_from_holiday_flag(
    full_index: pd.DatetimeIndex,
    holiday_flag: pd.Series
) -> pd.Series:
    hol = holiday_flag.reindex(full_index).fillna(0).astype(int)
    cny = pd.Series(0, index=full_index, dtype=int)

    i = 0
    while i < len(hol) - 1:
        if hol.iloc[i] == 1 and hol.iloc[i + 1] == 1:
            cny.iloc[i:i+2] = 1
            i += 2
        else:
            i += 1

    cny.name = "cny_flag"
    return cny

def build_cgh_cny_post_window(
    full_index: pd.DatetimeIndex,
    cny_flag: pd.Series,
    post_days: int = 4
) -> pd.Series:
    post = pd.Series(0, index=full_index, dtype=int)
    idx = np.where(cny_flag.values == 1)[0]

    if idx.size > 0:
        breaks = np.where(np.diff(idx) != 1)[0]
        block_ends = np.r_[breaks, len(idx) - 1]

        for be in block_ends:
            end_pos = idx[be]
            post.iloc[end_pos + 1 : min(end_pos + post_days + 1, len(post))] = 1

    post.name = "cny_post_window"
    return post

cgh_cny_flag = build_cgh_cny_flag_from_holiday_flag(
    cgh_full_index, cgh_public_holiday_flag
)

cgh_cny_post_window = build_cgh_cny_post_window(
    cgh_full_index, cgh_cny_flag, post_days=4
)

In [5]:
# CSV LOADING + ROBUST DATETIME PARSING
def try_parse_datetime(series, fmt=None, dayfirst=False):
    return pd.to_datetime(series, format=fmt, errors="coerce", dayfirst=dayfirst)

def parse_cgh_admit_datetime(df):
    s = df["A&E Admit Date Time"]
    dt = pd.Series(pd.NaT, index=df.index, dtype="datetime64[ns]")

    cand = try_parse_datetime(s, "%m/%d/%Y %I:%M:%S %p", dayfirst=False)
    dt = dt.fillna(cand)

    cand = try_parse_datetime(s, "%m/%d/%Y %I:%M %p", dayfirst=False)
    dt = dt.fillna(cand)

    cand = try_parse_datetime(s, "%d/%m/%Y %H:%M:%S", dayfirst=True)
    dt = dt.fillna(cand)

    cand = try_parse_datetime(s, "%d/%m/%Y %H:%M", dayfirst=True)
    dt = dt.fillna(cand)

    cand = pd.to_datetime(s, errors="coerce", dayfirst=True, infer_datetime_format=True)
    dt = dt.fillna(cand)

    cand = pd.to_datetime(s, errors="coerce", dayfirst=False, infer_datetime_format=True)
    dt = dt.fillna(cand)

    if "A&E Admit Date" in df.columns:
        d_only = pd.to_datetime(df["A&E Admit Date"], errors="coerce", dayfirst=True)
        dt = dt.where(dt.notna(), d_only)

    return dt

# Merge into AnE_Data_fin.csv
def build_unified_cgh_arrivals_csv(
    csv_2018_2024_path,
    csv_2025_path,
    output_csv_path
):
    df_2018_2024 = pd.read_csv(csv_2018_2024_path, low_memory=False)
    df_2025 = pd.read_csv(csv_2025_path, low_memory=False)

    df_all = pd.concat([df_2018_2024, df_2025], ignore_index=True)

    if "A&E Discharge Type Description" in df_all.columns:
        df_all = df_all[df_all["A&E Discharge Type Description"] != "Cancellation"].copy()

    df_all.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

# Build the unified file
build_unified_cgh_arrivals_csv(
    CGH_ER_ARRIVALS_2018_2024_CSV_PATH,
    CGH_ER_ARRIVALS_2025_CSV_PATH,
    CGH_ER_ARRIVALS_CSV_PATH
)

def load_cgh_daily_arrivals_from_csv(csv_path, start_date, end_date):
    df = pd.read_csv(csv_path, low_memory=False)

    if "A&E Triage Class (Computed)" in df.columns:
        df = df[df["A&E Triage Class (Computed)"] != "P4"].copy()

    dt = parse_cgh_admit_datetime(df)
    df = df.assign(dt=dt).dropna(subset=["dt"])

    df["day"] = pd.to_datetime(df["dt"], errors="coerce").dt.floor("D")
    df = df.dropna(subset=["day"])

    start_ts = pd.Timestamp(start_date)
    end_ts = pd.Timestamp(end_date)
    df = df[(df["day"] >= start_ts) & (df["day"] <= end_ts)].copy()

    daily_counts = df.groupby("day", sort=True).size()

    # Ensure strictly sorted by day
    daily_counts = daily_counts.sort_index()

    full_index = pd.date_range(start=start_ts, end=end_ts, freq="D")
    full_index = pd.to_datetime(full_index).tz_localize(None).normalize()

    s = daily_counts.reindex(full_index, fill_value=0).astype(float)
    s.index = pd.to_datetime(s.index).tz_localize(None).normalize()
    s.index.name = "Date"
    s.name = "Arrivals"

    return s

In [6]:
# LOAD TRAIN/TEST
cgh_train_arrivals = load_cgh_daily_arrivals_from_csv(
    CGH_ER_ARRIVALS_CSV_PATH,
    CGH_TRAIN_START_DATE,
    CGH_TRAIN_END_DATE
)

cgh_test_arrivals = load_cgh_daily_arrivals_from_csv(
    CGH_ER_ARRIVALS_CSV_PATH,
    CGH_TEST_START_DATE,
    CGH_TEST_END_DATE
)

cgh_arrivals_full = pd.concat([cgh_train_arrivals, cgh_test_arrivals]).sort_index()
cgh_arrivals_full.index.name = "Date"
cgh_arrivals_df = cgh_arrivals_full.to_frame()

In [7]:
# EVENT FEATURES: F1 + CONCERT 
def build_cgh_f1_event_df():
    years = [2022, 2023, 2024, 2025]
    all_races = []

    for year in years:
        df_f1_year = fastf1.get_event_schedule(year)
        df_f1_year = df_f1_year[df_f1_year["Country"] == "Singapore"]
        df_f1_year = df_f1_year[["EventDate", "Session1Date"]]
        all_races.append(df_f1_year)

    # SAFETY: if nothing returned, return empty event df
    if len(all_races) == 0:
        return pd.DataFrame(columns=["ds", "f1_event"])

    df_f1 = pd.concat(all_races, ignore_index=True)

    # SAFETY: if Singapore filter produced empty df
    if df_f1.empty:
        return pd.DataFrame(columns=["ds", "f1_event"])

    df_f1["Session1Date"] = pd.to_datetime(df_f1["Session1Date"]).dt.tz_localize(None).dt.normalize()
    df_f1["EventDate"]    = pd.to_datetime(df_f1["EventDate"]).dt.tz_localize(None).dt.normalize()

    def get_f1_dates(row):
        return pd.date_range(start=row["Session1Date"], end=row["EventDate"])

    f1_dates = df_f1.apply(get_f1_dates, axis=1)
    f1_dates = pd.DataFrame(f1_dates.explode(), columns=["ds"])
    f1_dates["ds"] = pd.to_datetime(f1_dates["ds"]).dt.tz_localize(None).dt.normalize()
    f1_dates["f1_event"] = 1
    return f1_dates

# F1 series 
try:
    cgh_f1_df = build_cgh_f1_event_df().set_index("ds")
    f1_series = cgh_f1_df.reindex(cgh_full_index)["f1_event"].fillna(0).astype(int)
except Exception:
    f1_series = pd.Series(0, index=cgh_full_index, dtype=int)

temp_events = f1_series.replace(0, np.nan)
cgh_f1_event_window = (
    temp_events
    .bfill(limit=4)
    .ffill(limit=2)
    .fillna(0)
    .astype(int)
)

# National Stadium sold out concerts only
def build_cgh_concert_event_df():
    concert_dates = [
        "2022-12-17", "2022-12-18",
        "2023-05-13", "2023-05-14",
        "2024-01-23", "2024-01-24", "2024-01-26", "2024-01-27", "2024-01-30", "2024-01-31",
        "2024-02-16",
        "2024-03-02", "2024-03-03", "2024-03-04", "2024-03-07", "2024-03-08", "2024-03-09",
        "2024-04-03", "2024-04-05", "2024-04-06",
        "2024-10-11", "2024-10-12", "2024-10-13",
        "2025-01-11", "2025-01-12",
        "2025-01-25", "2025-01-26",
        "2025-02-14", "2025-02-15",
        "2025-03-01",
        "2025-05-18", "2025-05-19", "2025-05-21", "2025-05-24"
    ]
    ds = pd.to_datetime(concert_dates).tz_localize(None).normalize()
    df_con = pd.DataFrame({"ds": ds})
    df_con["concert_event"] = 1
    return df_con

cgh_concert_df = build_cgh_concert_event_df().set_index("ds")
concert_series = cgh_concert_df.reindex(cgh_full_index)["concert_event"].fillna(0).astype(int)

temp_con = concert_series.replace(0, np.nan)
cgh_concert_event_window = (
    temp_con
    .bfill(limit=1)
    .ffill(limit=2)
    .fillna(0)
    .astype(int)
)

req         WARNING 	DEFAULT CACHE ENABLED! (112.0 KB) C:\Users\kchas\AppData\Local\Temp\fastf1


In [8]:
# EXOG MATRIX
cgh_temporal_exog_df = pd.DataFrame(index=cgh_full_index)

cgh_temporal_exog_df["dow"]   = cgh_temporal_exog_df.index.dayofweek
cgh_temporal_exog_df["dom"]   = cgh_temporal_exog_df.index.day
cgh_temporal_exog_df["qtr"]   = cgh_temporal_exog_df.index.quarter

cgh_temporal_exog_df["holiday"] = cgh_public_holiday_flag.reindex(cgh_full_index).fillna(0).astype(int).values
cgh_temporal_exog_df["holiday_post_window_2d"] = cgh_holiday_post_window_2d.reindex(cgh_full_index).fillna(0).astype(int).values
cgh_temporal_exog_df["cny_post_window"] = cgh_cny_post_window.reindex(cgh_full_index).fillna(0).astype(int).values

# dom_norm
cgh_temporal_exog_df["dom_norm"] = (
    (cgh_temporal_exog_df["dom"] - cgh_temporal_exog_df["dom"].mean())
    / (cgh_temporal_exog_df["dom"].std() + 1e-8)
)

# Holiday window
temp_hol = cgh_temporal_exog_df["holiday"].replace(0, np.nan)
cgh_temporal_exog_df["holiday_window"] = (
    temp_hol.bfill(limit=2).ffill(limit=2).fillna(0).astype(int)
)

# Event windows
cgh_temporal_exog_df["f1_event"] = (
    pd.Series(cgh_f1_event_window, index=cgh_full_index)
    .reindex(cgh_full_index)
    .fillna(0)
    .astype(int)
    .values
)

cgh_temporal_exog_df["concert_event"] = (
    pd.Series(cgh_concert_event_window, index=cgh_full_index)
    .reindex(cgh_full_index)
    .fillna(0)
    .astype(int)
    .values
)

# dummies
dow_dummies   = pd.get_dummies(cgh_temporal_exog_df["dow"], prefix="dow", drop_first=True)
qtr_dummies   = pd.get_dummies(cgh_temporal_exog_df["qtr"], prefix="qtr", drop_first=True)

cgh_exog_full = pd.concat(
    [
        dow_dummies,
        qtr_dummies,
        cgh_temporal_exog_df[
            [
                "dom_norm",
                "holiday",
                "holiday_window",
                "holiday_post_window_2d",
                "cny_post_window",
                "f1_event",
                "concert_event",
            ]
        ],
    ],
    axis=1
)

cgh_exog_full = (
    cgh_exog_full.apply(pd.to_numeric, errors="coerce")
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0.0)
    .astype("float64")
)

# align exog to arrivals df (index is 2022–2025)
cgh_exog_full = cgh_exog_full.reindex(cgh_arrivals_df.index).fillna(0.0)

# Train/Test slices
cgh_train_df = cgh_arrivals_df.loc[CGH_TRAIN_START_DATE:CGH_TRAIN_END_DATE]
cgh_test_df  = cgh_arrivals_df.loc[CGH_TEST_START_DATE:CGH_TEST_END_DATE]

cgh_y_train = cgh_train_df["Arrivals"].values.astype(float)
cgh_y_test  = cgh_test_df["Arrivals"].values.astype(float)

cgh_train_exog = cgh_exog_full.loc[cgh_train_df.index]
cgh_test_exog  = cgh_exog_full.loc[cgh_test_df.index]

cgh_test_index = cgh_test_df.index

if np.all(cgh_y_train == 0) or np.all(cgh_y_test == 0):
    raise ValueError("Train or test arrivals are all zeros. Check inputs.")

In [9]:
# Evaluation METRICS + SARIMAX REFIT/FORECAST
def compute_directional_accuracy_window(y_true_vec, y_pred_vec):
    yt = np.asarray(y_true_vec, float)
    yp = np.asarray(y_pred_vec, float)
    if len(yt) < 2 or len(yp) < 2:
        return np.nan
    return (np.sign(np.diff(yt)) == np.sign(np.diff(yp))).mean() * 100.0

def compute_mape_percent(y_true_vec, y_pred_vec):
    yt = np.asarray(y_true_vec, float)
    yp = np.asarray(y_pred_vec, float)
    mask = (yt != 0) & ~np.isnan(yt) & ~np.isnan(yp)
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((yt[mask] - yp[mask]) / yt[mask])) * 100.0

def compute_global_rmse_from_errors(err_vec):
    err = np.asarray(err_vec, float)
    err = err[~np.isnan(err)]
    if err.size == 0:
        return np.nan
    return float(np.sqrt(np.mean(err ** 2)))

def refit_cgh_sarimax_and_forecast(history_y, history_exog, steps, future_exog):
    model = SARIMAX(
        history_y,
        exog=history_exog,
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, CGH_WEEKLY_SEASONAL_PERIOD),
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    res = model.fit(disp=False)
    yhat = np.asarray(res.get_forecast(steps=steps, exog=future_exog).predicted_mean, dtype=float)
    return np.clip(yhat, 0.0, None)

In [10]:
# WALK-FORWARD EVALUATION
cgh_test_length = len(cgh_y_test)
cgh_max_horizon = max(CGH_FORECAST_HORIZON_DAYS)  # should be 3

# Storage for table 
cgh_preds_by_horizon = {
    h: np.full(cgh_test_length, np.nan, dtype=float)
    for h in CGH_FORECAST_HORIZON_DAYS
}

# Storage for metrics
cgh_walk_forward_results = {
    h: {
        "sq_err": [],    
        "abs_err": [],    
        "mape": [],       
        "da": []          
    }
    for h in CGH_FORECAST_HORIZON_DAYS
}

t0 = time.time()

# Main walk-forward (origin i = 0..N-1)
for i in range(cgh_test_length):

    # build history up to i-1 (same as your logic)
    history_y = np.r_[cgh_y_train, cgh_y_test[:i]]
    if i > 0:
        history_exog = np.vstack([cgh_train_exog.values, cgh_test_exog.values[:i]])
    else:
        history_exog = cgh_train_exog.values

    # we can only forecast up to what's left in test
    steps = min(cgh_max_horizon, cgh_test_length - i)
    if steps <= 0:
        continue

    future_exog = cgh_test_exog.values[i:i + steps]

    try:
        preds = refit_cgh_sarimax_and_forecast(history_y, history_exog, steps, future_exog)
    except Exception as e:
        if i == 0:
            print("First refit/forecast error:", repr(e))
            print("history_y shape:", history_y.shape)
            print("history_exog shape:", np.asarray(history_exog).shape)
            print("future_exog shape:", np.asarray(future_exog).shape)
        continue

    # Fill the prediction table
    for h in CGH_FORECAST_HORIZON_DAYS:
        target_idx = i + h - 1
        if target_idx < cgh_test_length and h <= steps:
            cgh_preds_by_horizon[h][target_idx] = preds[h - 1]

    # Evaluation matrix
    for h in CGH_FORECAST_HORIZON_DAYS:
        if h > steps:
            continue

        true_h = cgh_y_test[i:i + h]
        pred_h = preds[:h]

        if np.isnan(true_h).any() or np.isnan(pred_h).any():
            continue

        err = (true_h - pred_h)

        # RMSE and MAE
        cgh_walk_forward_results[h]["sq_err"].extend(list(err ** 2))
        cgh_walk_forward_results[h]["abs_err"].extend(list(np.abs(err)))

        # MAPE
        cgh_walk_forward_results[h]["mape"].append(compute_mape_percent(true_h, pred_h))

        # DA
        if h == 1:
            # direction vs previous actual 
            if i >= 1:
                prev_actual = cgh_y_test[i - 1]
                dir_true = np.sign(true_h[0] - prev_actual)
                dir_pred = np.sign(pred_h[0] - prev_actual)
                cgh_walk_forward_results[h]["da"].append(100.0 if dir_true == dir_pred else 0.0)
        else:
            # directional accuracy inside the h-length window
            cgh_walk_forward_results[h]["da"].append(compute_directional_accuracy_window(true_h, pred_h))

    if i % 30 == 0:
        gc.collect()

elapsed = time.time() - t0        

# Print summary
print("\nCGH ER SARIMAX Temporal Exogenous — Walk Forward Evaluation")
print(f"Train: {CGH_TRAIN_START_DATE} to {CGH_TRAIN_END_DATE} | Test: {CGH_TEST_START_DATE} to {CGH_TEST_END_DATE}")

for hrs, h in zip(CGH_FORECAST_HOURS, CGH_FORECAST_HORIZON_DAYS):
    sq = np.asarray(cgh_walk_forward_results[h]["sq_err"], float)
    ae = np.asarray(cgh_walk_forward_results[h]["abs_err"], float)
    mp = np.asarray(cgh_walk_forward_results[h]["mape"], float)
    da = np.asarray(cgh_walk_forward_results[h]["da"], float)

    rmse = np.sqrt(np.mean(sq)) if sq.size else np.nan
    mae  = np.mean(ae) if ae.size else np.nan
    mape = np.nanmean(mp) if mp.size else np.nan
    da_m = np.nanmean(da) if da.size else np.nan

    print(
        f"{hrs}h (~{h} day) → RMSE {rmse:.2f} | "
        f"MAE {mae:.2f} | "
        f"MAPE {mape:.2f}% | "
        f"DA {da_m:.2f}% "
    )

print(f"\nRuntime: {elapsed:.2f} seconds")


CGH ER SARIMAX Temporal Exogenous — Walk Forward Evaluation
Train: 2022-01-01 to 2024-12-31 | Test: 2025-01-01 to 2025-09-30
24h (~1 day) → RMSE 24.13 | MAE 18.53 | MAPE 4.92% | DA 83.09% 
48h (~2 day) → RMSE 24.23 | MAE 18.72 | MAPE 4.98% | DA 76.84% 
72h (~3 day) → RMSE 24.40 | MAE 18.81 | MAPE 5.00% | DA 76.94% 

Runtime: 3565.26 seconds


In [11]:
# MULTI-HORIZON PREDICTION TABLE
cgh_multi_horizon_table = pd.DataFrame(
    {"Date": cgh_test_index, "Actual_Arrivals": cgh_y_test}
)

cgh_multi_horizon_table["Predicted_Arrivals_1d"] = cgh_preds_by_horizon[1]
cgh_multi_horizon_table["Predicted_Arrivals_2d"] = cgh_preds_by_horizon[2]
cgh_multi_horizon_table["Predicted_Arrivals_3d"] = cgh_preds_by_horizon[3]

cgh_multi_horizon_table["Predicted_Arrivals_1d"] = cgh_multi_horizon_table["Predicted_Arrivals_1d"].round(2)
cgh_multi_horizon_table["Predicted_Arrivals_2d"] = cgh_multi_horizon_table["Predicted_Arrivals_2d"].round(2)
cgh_multi_horizon_table["Predicted_Arrivals_3d"] = cgh_multi_horizon_table["Predicted_Arrivals_3d"].round(2)

cgh_multi_horizon_table.to_csv(CGH_MULTI_HORIZON_CSV, index=False, encoding="utf-8-sig")
print(f"\nSaved: {CGH_MULTI_HORIZON_CSV}")
print(cgh_multi_horizon_table.head(10))
print(cgh_multi_horizon_table.tail(10))


Saved: CGH_ER_MultiHorizon_Predictions_Daily_Train2022to2024_Test2025_JnToSep.csv
        Date  Actual_Arrivals  Predicted_Arrivals_1d  Predicted_Arrivals_2d  \
0 2025-01-01            397.0                 359.39                    NaN   
1 2025-01-02            436.0                 417.52                 403.31   
2 2025-01-03            409.0                 418.27                 413.91   
3 2025-01-04            352.0                 356.33                 362.99   
4 2025-01-05            354.0                 362.39                 359.38   
5 2025-01-06            461.0                 464.21                 465.54   
6 2025-01-07            409.0                 417.07                 422.01   
7 2025-01-08            410.0                 406.18                 408.74   
8 2025-01-09            357.0                 402.84                 397.92   
9 2025-01-10            379.0                 411.60                 420.47   

   Predicted_Arrivals_3d  
0                   

In [14]:
# PRODUCTION MODEL PACKAGING 
cgh_exog_prod = cgh_exog_full.reindex(cgh_arrivals_full.index).fillna(0.0)

cgh_er_prod_model = SARIMAX(
    cgh_arrivals_full.values.astype(float),
    exog=cgh_exog_prod.values,
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1, CGH_WEEKLY_SEASONAL_PERIOD),
    enforce_stationarity=False,
    enforce_invertibility=False
)

cgh_er_prod_results = cgh_er_prod_model.fit(disp=False)

last_observed_date = (
    pd.to_datetime(cgh_arrivals_full.index.max())
    .tz_localize(None)
    .normalize()
)

cgh_er_model_bundle = {
    # model
    "sarimax_results": cgh_er_prod_results,

    # exog metadata
    "exogenous_feature_columns": list(cgh_exog_full.columns),

    # normalization stats
    "dom_mean": float(cgh_temporal_exog_df["dom"].mean()),
    "dom_std": float(cgh_temporal_exog_df["dom"].std()),

    # holiday logic 
    "holiday_pre_days": 0,
    "holiday_post_days": 2,
    "cny_post_days": 4,

    # training info
    "train_start": CGH_TRAIN_START_DATE,
    "train_end": CGH_TRAIN_END_DATE,
    "full_fit_start": CGH_FULL_START_DATE,
    "full_fit_end": CGH_FULL_END_DATE,

    # anchor
    "last_observed_date": str(last_observed_date),
}

with open(CGH_PROD_MODEL_PKL, "wb") as f:
    pickle.dump(cgh_er_model_bundle, f)

print(f"Saved CGH ER production model bundle: {CGH_PROD_MODEL_PKL}")
print(f"Last observed date saved in bundle: {last_observed_date}")

Saved CGH ER production model bundle: CGH_ER_SARIMAX_TemporalExog_ProdModel_FullFit_2022to2025.pkl
Last observed date saved in bundle: 2025-09-30 00:00:00


In [13]:
#Feature extraction
def get_param_map(sarimax_results):
    names = list(getattr(sarimax_results, "param_names", []))
    vals = np.asarray(getattr(sarimax_results, "params", []), dtype=float)

    if len(names) != len(vals):
        try:
            s = pd.Series(sarimax_results.params, index=sarimax_results.param_names)
            return s.to_dict()
        except Exception:
            raise ValueError(f"Cannot map param_names to params (names={len(names)} vs vals={len(vals)}).")

    return dict(zip(names, vals))

def describe_feature(col_name: str) -> str:
    FEATURE_DESCRIPTIONS = {
        "qtr_2": "Quarter  (Q2 vs Q1 baseline).",
        "qtr_3": "Quarter  (Q3 vs Q1 baseline).",
        "qtr_4": "Quarter  (Q4 vs Q1 baseline).",
        "holiday": "Public holiday flag (1=holiday).",
        "holiday_window": "Holiday +/-2-day window.",
        "holiday_post_window_2d": "1 for +1 to +2 days after a holiday (holiday day=0).",
        "cny_post_window": "1 for +1 to +4 days after detected 2-day CNY block (CNY days=0).",
        "f1_event": "Singapore F1 event window.",
        "concert_event": "Sold-out concert event window.",
    }
    if col_name.startswith("dow_"):
        return "Day-of-week."
    return FEATURE_DESCRIPTIONS.get(col_name, "Exogenous feature.")

def build_ranked_feature_table(model_bundle):
    res = model_bundle["sarimax_results"]
    exog_cols = list(model_bundle["exogenous_feature_columns"])

    param_names = list(getattr(res, "param_names", []))
    params = np.asarray(getattr(res, "params", []), dtype=float)

    # If names already match
    param_map = get_param_map(res)
    direct_hits = sum([1 for c in exog_cols if c in param_map])

    # If not, it’s probably x1..xK naming -> map by position
    if direct_hits == 0:
        # Collect any "x1", "x2", ... params in order
        x_like = []
        for i, n in enumerate(param_names):
            if re.match(r"^x\d+$", str(n).strip(), flags=re.IGNORECASE):
                x_like.append((i, n))

        # If statsmodels used generic x-params, it should equal number of exog columns
        if len(x_like) >= len(exog_cols):
            # take first K x-params and map to your exog column order
            x_idx = [i for i, _ in x_like[:len(exog_cols)]]
            exog_param_vals = params[x_idx]

            rows = []
            for col, coef in zip(exog_cols, exog_param_vals):
                rows.append({
                    "column_name": col,
                    "description": describe_feature(col),
                    "coefficient": float(coef),
                    "abs_coefficient": abs(float(coef))
                })

            df = pd.DataFrame(rows)

        else:
            # fallback: try to find params that start with "exog" or contain your col name
            rows = []
            for col in exog_cols:
                coef = np.nan
                # try exact
                if col in param_map:
                    coef = float(param_map[col])
                else:
                    # try partial match
                    for n in param_names:
                        if str(col) in str(n):
                            coef = float(param_map.get(n, np.nan))
                            break
                rows.append({
                    "column_name": col,
                    "description": describe_feature(col),
                    "coefficient": coef,
                    "abs_coefficient": abs(coef) if not np.isnan(coef) else np.nan
                })
            df = pd.DataFrame(rows)

    else:
        # normal case (names match)
        rows = []
        for col in exog_cols:
            coef = float(param_map.get(col, np.nan))
            rows.append({
                "column_name": col,
                "description": describe_feature(col),
                "coefficient": coef,
                "abs_coefficient": abs(coef) if not np.isnan(coef) else np.nan
            })
        df = pd.DataFrame(rows)

    # rank + sort by importance
    df = df.sort_values("abs_coefficient", ascending=False, na_position="last").reset_index(drop=True)
    df.insert(0, "rank", np.arange(1, len(df) + 1))
    df["coefficient"] = df["coefficient"].round(6)
    df = df.drop(columns=["abs_coefficient"])

    return df

from tabulate import tabulate
import pandas as pd

def print_boxed_table(df: pd.DataFrame):
    df_fmt = df.copy()

    # format coefficient nicely
    df_fmt["coefficient"] = df_fmt["coefficient"].map(
        lambda x: f"{x:,.6f}" if pd.notna(x) else "NA"
    )

    print(tabulate(df_fmt, headers="keys", tablefmt="grid", showindex=False))

# ---- Usage ----
coef_df = build_ranked_feature_table(cgh_er_model_bundle)
print_boxed_table(coef_df)

+--------+------------------------+------------------------------------------------------------------+---------------+
|   rank | column_name            | description                                                      |   coefficient |
+========+========================+==================================================================+===============+
|      1 | holiday                | Public holiday flag (1=holiday).                                 |    -39.0334   |
+--------+------------------------+------------------------------------------------------------------+---------------+
|      2 | holiday_post_window_2d | 1 for +1 to +2 days after a holiday (holiday day=0).             |     20.657    |
+--------+------------------------+------------------------------------------------------------------+---------------+
|      3 | qtr_2                  | Quarter  (Q2 vs Q1 baseline).                                    |     19.4921   |
+--------+------------------------+-------------